## TÖL403G Lokaverkefni

In [ ]:
import random, time, math
from dataclasses import dataclass
from heapdict import heapdict
from ast import literal_eval

Byrjum á því að útfæra einfalda gagnagrind fyrir net. Þetta mun halda utan um upplýsingarnar í skránum ásamt því að gera okkur kleift að útfæra aðferðir sem munu vera gagnlegar síðar.

In [35]:
@dataclass
class Node:
    id: int
    x_coord: float
    y_coord: float
    primary: bool

@dataclass
class Edge:
    node_id_from: int
    node_id_to: int
    length: float
    name: list[str]

class Graph:

    def __init__(self):
        self.nodes: dict[int, Node] = {}
        self.edges: dict[tuple[int, int], list[Edge]] = {}
        self.adj_list: dict[int, list[int]] = {}

    def add_node(self, node: Node) -> None:
        id = node.id
        if id not in self.nodes:
            self.nodes[id] = node

    def add_edge(self, edge: Edge) -> None:
        id_from = edge.node_id_from
        id_to = edge.node_id_to
        if id_from in self.nodes and id_to in self.nodes:
            self.edges.setdefault((id_from, id_to), []).append(edge)
            self.adj_list.setdefault(id_from, []).append(id_to)
        else:
            print(f'skipped edge missing node(s) {id_from} -> {id_to}')


**2.3.1 Þáttun(*)**

Lesið inn netið úr skránum sem eru gefnar, nodes.tsv og edges.tsv. Í skránni nodes.tsv
eru hnútar með auðkenni (id), hnit (x, y) og hvort þeir séu á aðalvegi (primary). Í skránni
edges.tsv eru leggi frá hnúti u til hnúts v með lengd/length, mæld í metrum, og nafn
(name).

In [42]:
graph = Graph()

with open('nodes.tsv', 'r', encoding='utf-8') as file:
    next(file) # skip header
    for line in file:
        parts = line.strip().split('\t')
        node = Node(
            id = int(parts[0]),
            x_coord = float(parts[1]),
            y_coord = float(parts[2]),
            primary = parts[3].lower() == 'true'
        )
        graph.add_node(node)

def parse_edge_name(name: str) -> list[str]:
    name = name.strip()
    if not name:
        return []
    
    if name.startswith('[') and name.endswith(']'):
        try:
            result = literal_eval(name)
            if isinstance(result, list[str]):
                return result
            else:
                return [str(result)]
        except Exception:
            return [name]
    return [name]

with open('edges.tsv', 'r', encoding='utf-8') as file:
    next(file) # skip header
    for line in file:
        parts = line.strip().split('\t')
        edge = Edge(
            node_id_from = int(parts[0]),
            node_id_to = int(parts[1]),
            length = float(parts[2]),
            name = parse_edge_name(parts[3]) if len(parts) > 3 else []
        )
        graph.add_edge(edge)

***2.3.2 Leit (⋆⋆)***

Ef við setjum hleðslustöðvar á hnúta $v_1, . . . , v_k$ þá er hægt að nota reikniriti Dijkstra til að
finna stystu fjarlægð frá hverjum hnúti $u$ í hleðslustöð $v_i$. Útfærið reikniritið sem tekur
inn lista af lokahnútum og reiknar fjarlægðir frá öllum hnútum í netinu. Athugið að netið
er stefnt net.

In [58]:
def Dijkstra(g: Graph, source_id: int, target_id: int = None) -> tuple[dict[int, float], dict[int, int]]:
    if not g.nodes:
        raise ValueError('Graph is empty')
    if source_id not in g.nodes:
        raise ValueError(f'source id: {source_id} not in graph')
    if target_id is not None and target_id not in g.nodes:
        raise ValueError(f'target id: {target_id} not in graph')

    dist = {}
    prev = {}
    Q = heapdict()
    dist[source_id] = 0
    Q[source_id] = dist[source_id]
    for v, _ in g.nodes.items():
        if v != source_id:
            dist[v] = float('inf')
            prev[v] = None
            Q[v] = dist[v]

    while Q:
        u, _ = Q.popitem()
        if target_id is not None and u == target_id:
            break

        for neighbour in g.adj_list.get(u, []):
            for edge in g.edges.get((u, neighbour), []):
                if neighbour in Q:
                    alt = dist[u] + edge.length
                    if alt < dist[neighbour]:
                        dist[neighbour] = alt
                        prev[neighbour] = u
                        Q[neighbour] = dist[neighbour]

    return (dist, prev)

def shortest_paths_to_charging_stations(g: Graph, V: list[int]) -> list[tuple[int, int, float]]: # [(u_i.id, v_i.id, path_length)]
    shortest_paths = []
    for u, _ in g.nodes.items():
        (dist, _) = Dijkstra(g, u)
        for v in V:
            if v in g.nodes:
                shortest_paths.append((u, v, dist[v]))
    return shortest_paths

***2.3.3 Framsetning (⋆)***

Setjið fimm hleðslustöðvar í netið og sýnið stystu leið fyrir fimm punkta og teiknið upp á
kort. Tékkið ykkur af með því að bera saman leiðina sem er fundin og fjarlægðina miðað
við kortavefi eins og t.d. Google Maps.

In [ ]:


    '''
    To display the path
    S = []
    u = target
    if prev[u.id] or u.id == s_id:
        while u:
            S.insert(0, u)
            u = prev[u]
    '''

***2.3.4 Tímamælingar (⋆)***

Mælið tímann sem reiknirit Dijkstra tekur að reikna allar fjarlægðir í netinu með fimm
hleðslustöðvum.

In [59]:
charging_stations = random.sample(list(graph.nodes.keys()), 5)

start_time = time.time()
shortest_paths_to_charging_stations(graph, charging_stations)
end_time = time.time()
elapsed_time = end_time - start_time
print(elapsed_time / 60)

16.950663336118062


***2.3.5 $A^*$ reikniritið (⋆⋆)***

Útfærið $A^*$ reikniritið sem tekur inn lista af lokahnútum og reiknar fjarlægðir frá öll-
um hnútum í netinu. Sem neðra mat á fjarlægð á milli hnútanna má taka $d(u, v) =\sqrt{(x_u − x_v )^2 + (y_u − y_v )^2}$, þ.e. beina loftlínu milli punktanna. Mælið tíma og berið saman
við reiknirit Dijkstra.

In [ ]:
def potential(u_coords: tuple[float, float], v_coords: tuple[float, float]) -> float:
    (x_u, y_u) = u_coords
    (x_v, y_v) = v_coords
    return math.sqrt((x_u - x_v)**2 + (y_u - y_v)**2)

def A_star(g: Graph, source: Node, goal: Node):
    return 0